In [ ]:
print("=== Heap Diagnostic Code Loaded ===")

class Heap:
    def __init__(self, size):
        self.blocks = [(0, size, True)]  # (start, size, free)

    def dump(self):
        print("Heap:", self.blocks)

    def malloc(self, size):
        print(f"\nCALL: malloc({size})")

        for i, (start, bsize, free) in enumerate(self.blocks):
            if free and bsize >= size:
                self.blocks[i] = (start, size, False)
                if bsize > size:
                    self.blocks.insert(i + 1, (start + size, bsize - size, True))
                print("ALLOCATED")
                return start

        # ---- FORCED FAILURE PATH ----
        free_blocks = [bsize for _, bsize, free in self.blocks if free]
        total_free = sum(free_blocks)
        largest = max(free_blocks) if free_blocks else 0

        print(">>> ALLOCATION FAILED <<<")
        print("TOTAL FREE :", total_free)
        print("LARGEST BLK:", largest)
        print("REQUESTED :", size)

        if total_free >= size:
            raise RuntimeError("FRAGMENTATION ERROR DETECTED")
        else:
            raise RuntimeError("OUT OF MEMORY")

    def free(self, ptr):
        print(f"\nCALL: free({ptr})")
        for i, (start, size, free) in enumerate(self.blocks):
            if start == ptr and not free:
                self.blocks[i] = (start, size, True)
                print("FREED")
                return
        raise RuntimeError("INVALID FREE")


=== Heap Diagnostic Code Loaded ===


In [ ]:
heap = Heap(100)
heap.dump()

p1 = heap.malloc(20)
heap.dump()

p2 = heap.malloc(30)
heap.dump()

heap.free(p1)
heap.dump()

heap.free(p2)
heap.dump()

Heap: [(0, 100, True)]

CALL: malloc(20)
ALLOCATED
Heap: [(0, 20, False), (20, 80, True)]

CALL: malloc(30)
ALLOCATED
Heap: [(0, 20, False), (20, 30, False), (50, 50, True)]

CALL: free(0)
FREED
Heap: [(0, 20, True), (20, 30, False), (50, 50, True)]

CALL: free(20)
FREED
Heap: [(0, 20, True), (20, 30, True), (50, 50, True)]


In [ ]:
heap = Heap(100)

a = heap.malloc(20)   # 0–19
b = heap.malloc(20)   # 20–39
c = heap.malloc(20)   # 40–59
d = heap.malloc(20)   # 60–79
heap.dump()

heap.free(b)          # free 20–39
heap.free(d)          # free 60–79
heap.dump()

# Total free = 40
# Largest block = 20
heap.malloc(25)       # MUST FAIL (fragmentation)


CALL: malloc(20)
ALLOCATED

CALL: malloc(20)
ALLOCATED

CALL: malloc(20)
ALLOCATED

CALL: malloc(20)
ALLOCATED
Heap: [(0, 20, False), (20, 20, False), (40, 20, False), (60, 20, False), (80, 20, True)]

CALL: free(20)
FREED

CALL: free(60)
FREED
Heap: [(0, 20, False), (20, 20, True), (40, 20, False), (60, 20, True), (80, 20, True)]

CALL: malloc(25)
>>> ALLOCATION FAILED <<<
TOTAL FREE : 60
LARGEST BLK: 20
REQUESTED : 25


RuntimeError: FRAGMENTATION ERROR DETECTED

In [ ]:
heap = Heap(50)

p1 = heap.malloc(30)
p2 = heap.malloc(15)

heap.dump()

# Truly out of memory
p3 = heap.malloc(10)

In [ ]:
heap.free(999)   # Invalid pointer